<a href="https://colab.research.google.com/github/Ahmad162539/VMJ/blob/main/fgsm_attack_update_DEFENSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

import sys
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D  # Updated import statement
from tensorflow.keras.utils import plot_model
from keras import backend as K
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/T25 (1).csv')

In [ ]:
set(df['Dataset'].tolist())

In [ ]:
df.columns

In [ ]:
X = [eval(i) for i in df['resized_scaled'].tolist()]
X = np.array(X)
y = np.array(df['SOH'].tolist()) ##%changed

In [ ]:
# labeling for stratified random sampling
label = df['SOH'].tolist()
label_l = []
for cnt in range(len(label)):
    i = label[cnt]
    if i>=1.0 :
        lab = '1~'
        num = 0
    elif 0.98<=i<1.0 :
        lab = '0.98~1'
        num = 1
    elif 0.96<=i<0.98 :
        lab = '0.96~0.98'
        num = 2
    elif 0.94<=i<0.96 :
        lab = '0.94~0.96'
        num = 3
    elif 0.92<=i<0.94 :
        lab = '0.92~0.94'
        num = 4
    elif 0.90<=i<0.92 :
        lab = '0.90~0.92'
        num = 5

    elif 0.88<=i<0.9 :
        lab = '0.88~0.9'
        num = 6
    elif 0.86<=i<0.88 :
        lab = '0.86~0.88'
        num = 7
    elif 0.84<=i<0.86 :
        lab = '0.84~0.86'
        num = 8
    elif 0.82<=i<0.84 :
        lab = '0.82~0.84'
        num = 9
    elif 0.80<=i<0.82 :
        lab = '0.80~0.82'
        num = 10


    elif 0.78<=i<0.8 :
        lab = '0.78~0.8'
        num = 11
    elif 0.76<=i<0.78 :
        lab = '0.76~0.78'
        num = 12
    elif 0.74<=i<0.76 :
        lab = '0.74~0.76'
        num = 13
    elif 0.72<=i<0.74 :
        lab = '0.72~0.74'
        num = 14
    elif 0.70<=i<0.72 :
        lab = '0.70~0.72'
        num = 15

    else:
        lab = '~0.7'
        num = 16
    label_l.append(num)

In [ ]:
sss = StratifiedShuffleSplit(n_splits = 10, test_size = 0.3)
for train_idx, test_idx in sss.split(X, np.array(label_l)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = np.array(y)[train_idx], np.array(y)[test_idx]
    y_valid = np.array(label_l)[test_idx]

In [ ]:
sss = StratifiedShuffleSplit(n_splits = 10, test_size = 0.3)
for train_idx, test_idx1 in sss.split(X_test, y_valid):
    X_valid, X_test_f = X_test[train_idx], X_test[test_idx1]
    y_valid, y_test_f = np.array(y_test)[train_idx], np.array(y_test)[test_idx1]
X_valid.shape


In [ ]:
data_list = []
for i in df['Dataset'][test_idx[test_idx1]].tolist():
    if 'LFP' in i:
        data_list.append('LFP')
    if 'NCA' in i:
        data_list.append('NCA')
    if 'NMC' in i:
        data_list.append('NMC')

In [ ]:
x_train = X_train
x_valid = X_valid
x_test = X_test_f
x_test.shape
x_train.shape

In [ ]:
rows = len(X[0])
cols = 3
input_shape = (rows, cols, 1)
x_train = x_train.reshape(x_train.shape[0], rows, cols, 1)
x_test = x_test.reshape(x_test.shape[0], rows, cols, 1)
x_valid = x_valid.reshape(x_valid.shape[0], rows, cols, 1)

x_train = x_train.astype('float32') /255.0
x_test = x_test.astype('float32')/255.0
x_valid = x_valid.astype('float32')/255.0

batch_size = 64
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

In [ ]:
y_train

In [ ]:
###student model model with additional feature extraction branch
model = Sequential()
model.add(Conv2D(32, kernel_size = (2,2), strides = (1,1), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(Conv2D(64, kernel_size = (2,2), strides = (1,1), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1))
model.summary()
###

In [ ]:
start = time.time()
epochs = 5
print('start time : ' , start)
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mse'])
hist2 = model.fit(x_train, y_train, epochs = epochs, verbose = 1, validation_data = (x_valid, y_valid))

In [ ]:
y_vloss = hist2.history['val_loss']
y_loss = hist2.history['loss']
x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
plt.legend()
plt.title('epoch40_loss')

In [ ]:

def fgsm_perturb(x_test, epsilon):
    x_test_tf = tf.convert_to_tensor(x_test, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(x_test_tf)
        output = model(x_test_tf)
        loss = tf.keras.losses.mean_squared_error(y_test_f, output)
    # Calculate gradient of the loss with respect to the input
    gradient = tape.gradient(loss, x_test_tf)

    # Create perturbation using the sign of the gradient
    perturbation = epsilon * tf.sign(gradient)

    # Apply perturbation to the input data
    perturbed_data = x_test_tf + perturbation

    return perturbed_data.numpy()

# Assuming model and labels are defined elsewhere
#x_test_ad = fgsm_perturb(x_test, 0.01)
#x_test_ad.shape
x_test_ad = fgsm_perturb(x_test, 0.02)
#print("Original input:", x_test[0])
#print("Perturbed input:", x_test_ad[0])

# Evaluate the model on the original and perturbed data
original_predictions = model.predict(x_test, verbose = 0)
perturbed_predictions = model.predict(x_test_ad, verbose = 0)

# Compare predictions
#print("Original predictions:", original_predictions[0])
#print("Perturbed predictions:", perturbed_predictions[0])

# Continue with plotting or other analysis

In [ ]:
predict = model.predict(x_test_ad, verbose = 0)


In [ ]:
p = []
t = []
for i in range(len(predict)):
    p.append(predict[i])
    t.append(y_test_f[i])
    #print('predicted SOH: ' , predict[i],',' , 'Answer SOH: '  , y_test[i])

plt.scatter(y_test_f, predict)
plt.plot(y_test_f, y_test_f,c='red')
plt.show()

In [ ]:
r2 = metrics.r2_score(y_test_f, predict)
rmse = metrics.mean_squared_error(y_test_f, predict)**0.5
mse = metrics.mean_squared_error(y_test_f, predict)
mae = metrics.mean_absolute_error(y_test_f, predict)
mape = metrics.mean_absolute_percentage_error(y_test_f,predict)

In [ ]:
dic = {}
dic['r2'] = [r2]
dic['rmse'] = [rmse]
dic['mse'] = [mse]
dic['mae'] = [mae]
dic['mape'] = [mape]
d = pd.DataFrame(dic)
d


Defense

In [ ]:
from tensorflow.keras.models import Model
# Assuming x_train, y_train, x_valid, y_valid are your training and validation data

# Original teacher model definition
teacher_model = Sequential()
teacher_model.add(Conv2D(32, kernel_size=(2, 2), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
teacher_model.add(Conv2D(64, kernel_size=(2, 2), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
teacher_model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
teacher_model.add(Flatten())
teacher_model.add(Dense(64, activation='relu', name='feature_extraction'))
teacher_model.add(Dense(1))
teacher_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# Original student model definition
student_model = Sequential()
student_model.add(Conv2D(32, kernel_size=(2, 2), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
student_model.add(Conv2D(64, kernel_size=(2, 2), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
student_model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
student_model.add(Flatten())
student_model.add(Dense(64, activation='relu', name='feature_extraction'))
student_model.add(Dense(1))
student_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# Create a new model to extract features from the 'feature_extraction' layer
feature_extraction_model_teacher = Model(inputs=teacher_model.input, outputs=teacher_model.get_layer('feature_extraction').output)
feature_extraction_model_student = Model(inputs=student_model.input, outputs=student_model.get_layer('feature_extraction').output)

# Define custom loss function
def custom_loss(y_true, y_pred):
    # Extract features from the 'feature_extraction' layer for both teacher and student
    features_teacher = feature_extraction_model_teacher(x_train)
    features_student = feature_extraction_model_student(x_train)

    # Compute MSE between features of teacher and student
    mse_feature = tf.keras.losses.MeanSquaredError()(features_teacher, features_student)

    # Compute absolute error between the output of teacher and student
    abs_error_output = tf.keras.losses.MeanAbsoluteError()(teacher_model(x_train), student_model(x_train))

    # You can adjust the weights (alpha and beta) based on your preference
    alpha = 1
    beta = 1

    # Combined loss
    total_loss = alpha * mse_feature + beta * abs_error_output

    return total_loss

# Compile and train the student model with the custom loss function
student_model.compile(loss=custom_loss, optimizer='adam', metrics=['mse'])
epochs = 2
history = student_model.fit(x_train, y_train, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid))


In [ ]:
y_vloss = hist2.history['val_loss']
y_loss = hist2.history['loss']
x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
plt.legend()
plt.title('epoch40_loss')

In [ ]:
def fgsm_perturb(x_test, epsilon):
    x_test_tf = tf.convert_to_tensor(x_test, dtype=tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(x_test_tf)
        output = teacher_model(x_test_tf)
        loss = tf.keras.losses.mean_squared_error(y_test_f, output)
    # Calculate gradient of the loss with respect to the input
    gradient = tape.gradient(loss, x_test_tf)

    # Create perturbation using the sign of the gradient
    perturbation = epsilon * tf.sign(gradient)

    # Apply perturbation to the input data
    perturbed_data = x_test_tf + perturbation

    return perturbed_data.numpy()

# Assuming model and labels are defined elsewhere
#x_test_ad = fgsm_perturb(x_test, 0.01)
#x_test_ad.shape
x_test_ad = fgsm_perturb(x_test, 0.02)
#print("Original input:", x_test[0])
#print("Perturbed input:", x_test_ad[0])

# Evaluate the model on the original and perturbed data
original_predictions = teacher_model.predict(x_test, verbose = 0)
perturbed_predictions = teacher_model.predict(x_test_ad, verbose = 0)

# Compare predictions
#print("Original predictions:", original_predictions[0])
#print("Perturbed predictions:", perturbed_predictions[0])

# Continue with plotting or other analysis

In [ ]:
predict =  model.predict(x_test, verbose = 0)

In [ ]:
p = []
t = []
for i in range(len(predict)):
    p.append(predict[i])
    t.append(y_test_f[i])
    #print('predicted SOH: ' , predict[i],',' , 'Answer SOH: '  , y_test[i])

plt.scatter(y_test_f, predict)
plt.plot(y_test_f, y_test_f,c='red')
plt.show()

In [ ]:
r2 = metrics.r2_score(y_test_f, predict)
rmse = metrics.mean_squared_error(y_test_f, predict)**0.5
mse = metrics.mean_squared_error(y_test_f, predict)
mae = metrics.mean_absolute_error(y_test_f, predict)
mape = metrics.mean_absolute_percentage_error(y_test_f,predict)

In [ ]:
dic = {}
dic['r2'] = [r2]
dic['rmse'] = [rmse]
dic['mse'] = [mse]
dic['mae'] = [mae]
dic['mape'] = [mape]
d = pd.DataFrame(dic)

In [ ]:
d